# InSAR coherence timeseries

This notebook demonstrates how to generate InSAR coherence time series for all Sentinel-1 bursts defined in a given GeoJSON file (e.g., `s1_bursts_glacierSouthTyrol.geojson`). The GeoJSON file is a filtered subset of the Burst ID map available from the SAR-MPC products (https://sar-mpc.eu/sar-mpc-products/), containing information about subswath name and burst ID for each burst.

In this example, we generate a time series of 12 days InSAR coherence for the summer months of 2023, using all Sentinel-1 bursts that cover the glaciers of South Tyrol.

In [1]:
import openeo
import geopandas as gpd
import os

import urllib
import json
from datetime import datetime

## Input setting and preparation

In [2]:
start_date, end_date = ["2023-07-01", "2023-09-30"]

output_folder = '/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/coh_timseries/'

bursts_filename = 's1_bursts_glacierSouthTyrol.geojson'
bursts = gpd.read_file(bursts_filename)

In [3]:
url = "https://openeo.dataspace.copernicus.eu"
connection = openeo.connect(url).authenticate_oidc()

Authenticated using refresh token.


In [4]:
def create_InSARpairs(burst_id, subswath_id, temporal_baseline):

    https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
        f"ContentDate/Start ge {start_date}T00:00:00.000Z and ContentDate/Start le {end_date}T23:59:59.000Z and "
        f"PolarisationChannels eq 'VV' and "
        f"BurstId eq {burst_id} and SwathIdentifier eq '{subswath_id}'"
    ) + "&$top=1000"
    
    with urllib.request.urlopen(https_request) as response:
        content = response.read().decode()
        s1_bursts = json.loads(content)

    dates = [datetime.strptime(b['BeginningDateTime'][:10], "%Y-%m-%d") for b in s1_bursts['value']]
    dates.sort()
    InSARpairs = []
    for date_ref in dates:
        for date_sec in dates:
            if (date_ref - date_sec).days == -temporal_baseline:
                InSARpairs.append([
                    datetime.strftime(date_ref, "%Y-%m-%d"),
                    datetime.strftime(date_sec, "%Y-%m-%d")
                ])

    return InSARpairs

## Create and start all the jobs (one job for each burst)

SAR coherence parallel processing with InSAR pair list as input

In [5]:
#b = bursts.iloc[0]
for idx, b in bursts.iterrows():

    datacube = connection.datacube_from_process(
        process_id="sar_coherence_parallel",
        InSAR_pairs=create_InSARpairs(int(b['burst_id']), b['subswath_name'], 12),
        burst_id=int(b['burst_id']),
        polarization="vv",
        sub_swath=b['subswath_name'],
        coherence_window_rg=21,
        coherence_window_az=5,
    )
    datacube = datacube.save_result(format='GTiff')

    job = datacube.create_job(
        title=f"sar_coherence_T{b['relative_orbit_number']}_{b['subswath_name']}_{b['burst_id']}",
        job_options={"image-name": "python311-dev"}
    )
    job.start()

SAR coherence processing (no parallel) with temporal extent as input

In [8]:
# for idx, b in bursts.iterrows():

#     datacube = connection.datacube_from_process(
#         process_id="sar_coherence",
#         temporal_extent=[start_date, end_date],
#         temporal_baseline=12,
#         burst_id=int(b['burst_id']),
#         polarization="vv",
#         sub_swath=b['subswath_name'],
#         coherence_window_rg=21,
#         coherence_window_az=5,
#     )
#     datacube = datacube.save_result(format='GTiff')

#     job = datacube.create_job(
#         title=f"sar_coherence_T{b['relative_orbit_number']}_{b['subswath_name']}_{b['burst_id']}"
#     )
#     job.start()

Check job status

In [6]:
connection.list_jobs()

[{'created': '2026-01-20T07:28:48Z',
  'id': 'j-26012007284845528682d8b082ad3072',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T168_IW2_359502',
  'updated': '2026-01-20T07:32:16Z'},
 {'created': '2026-01-20T07:28:34Z',
  'id': 'j-260120072834473eb9d5cc5b1697d6c0',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T168_IW2_359501',
  'updated': '2026-01-20T07:32:16Z'},
 {'created': '2026-01-20T07:28:19Z',
  'id': 'j-260120072819403fa96b8348e094fd33',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T168_IW2_359500',
  'updated': '2026-01-20T07:32:17Z'},
 {'created': '2026-01-20T07:28:05Z',
  'id': 'j-26012007280543eebe11151bf5a79494',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T168_IW2_359499',
  'updated': '2026-01-20T07:32:17Z'},
 {'created': '2026-01-20T07:27:47Z',
  'id': 'j-26012007274748bdb1bcc641ec764de8',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T168_IW1_359499',
  'updated': '2026-01-20T07:32:17Z'},
 {'created': '2026-01-20T07:27:33Z',
  'id': 'j-2601200727334d1791427efb633a4f32',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T168_IW1_359498',
  'updated': '2026-01-20T07:32:17Z'},
 {'created': '2026-01-20T07:27:19Z',
  'id': 'j-2601200727194c819d143f2e6b5cee85',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T117_IW3_249436',
  'updated': '2026-01-20T07:32:18Z'},
 {'created': '2026-01-20T07:27:05Z',
  'id': 'j-2601200727054c09a2d44f68b870017d',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T117_IW2_249436',
  'updated': '2026-01-20T07:32:18Z'},
 {'created': '2026-01-20T07:26:45Z',
  'id': 'j-260120072645491f890e7830bd29aa9c',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T117_IW1_249436',
  'updated': '2026-01-20T07:32:18Z'},
 {'created': '2026-01-20T07:26:31Z',
  'id': 'j-26012007263144318923c35f1004a689',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T117_IW3_249435',
  'updated': '2026-01-20T07:32:19Z'},
 {'created': '2026-01-20T07:26:17Z',
  'id': 'j-2601200726174ed88bde21a8cc77e10e',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T117_IW2_249435',
  'updated': '2026-01-20T07:32:19Z'},
 {'created': '2026-01-20T07:26:03Z',
  'id': 'j-2601200726034e169d9684cadb6173e9',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T117_IW1_249435',
  'updated': '2026-01-20T07:32:19Z'},
 {'created': '2026-01-20T07:25:47Z',
  'id': 'j-26012007254748a4b9685e3fb2584aa3',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T117_IW2_249434',
  'updated': '2026-01-20T07:32:20Z'},
 {'created': '2026-01-20T07:25:33Z',
  'id': 'j-26012007253345d5aeebb1c0e8906345',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T117_IW1_249434',
  'updated': '2026-01-20T07:32:20Z'},
 {'created': '2026-01-20T07:25:20Z',
  'id': 'j-2601200725204062ab886aa4ad9159d9',
  'status': 'running',
  'title': 'sar_coherence_T95_IW3_202702',
  'updated': '2026-01-20T07:32:20Z'},
 {'created': '2026-01-20T07:24:58Z',
  'id': 'j-26012007245846e090dbbeb68fe12a72',
  'progress': 0,
  'status': 'queued',
  'title': 'sar_coherence_T95_IW3_202701',
  'updated': '2026-01-20T07:32:21Z'},
 {'created': '2026-01-20T07:24:43Z',
  'id': 'j-2601200724434b5598e30c59d1abb577',
  'status': 'running',
  'title': 'sar_coherence_T95_IW2_202701',
  'updated': '2026-01-20T07:32:21Z'},
 {'created': '2026-01-20T07:24:30Z',
  'id': 'j-2601200724304c8eadaa128a65f65f84',
  'status': 'running',
  'title': 'sar_coherence_T95_IW3_202700',
  'updated': '2026-01-20T07:32:21Z'},
 {'created': '2026-01-20T07:24:16Z',
  'id': 'j-2601200724164f25b2674244988cb793',
  'status': 'running',
  'title': 'sar_coherence_T95_IW2_202700',
  'updated': '2026-01-20T07:32:21Z'},
 {'created': '2026-01-20T07:24:02Z',
  'id': 'j-260120072402487bb31d3d95c9f1c941',
  'status': 'running',
  'title': 'sar_coherence_T44_IW1_92639',
  'updated': '2026-01-20T07:32:22Z'},
 {'crea

In [8]:
job_id = 'j-251209082325457ea3e8d47fa023802a'

In [13]:
connection.job(job_id)

<BatchJob job_id='j-251209082325457ea3e8d47fa023802a'>

In [9]:
connection.job(job_id).status()

'error'

In [14]:
connection.job(job_id).logs()

[{'id': '[1765269713435, 622442]',
  'time': '2025-12-09T08:41:53.435Z',
  'level': 'info',
  'message': 'Starting batch job os.getpid()=85: start 2025-12-09 08:41:53.435319'},
 {'id': '[1765269713436, 544334]',
  'time': '2025-12-09T08:41:53.436Z',
  'level': 'info',
  'message': "batch_job.py sys.version='3.11.7 (main, Aug 23 2024, 00:00:00) [GCC 11.4.1 20231218 (Red Hat 11.4.1-3)]'"},
 {'id': '[1765269713467, 709509]',
  'time': '2025-12-09T08:41:53.467Z',
  'level': 'info',
  'message': 'Loading custom processes from /opt/openeo-geopyspark-k8s-custom-processes/src/openeo_geopyspark_k8s_custom_processes/custom_processes.py'},
 {'id': '[1765269713469, 14298]',
  'time': '2025-12-09T08:41:53.469Z',
  'level': 'info',
  'message': "load_custom_processes: exec loaded '/opt/openeo-geopyspark-k8s-custom-processes/src/openeo_geopyspark_k8s_custom_processes/custom_processes.py'"},
 {'id': '[1765269713469, 347258]',
  'time': '2025-12-09T08:41:53.469Z',
  'level': 'info',
  'message': 'Overriding process sar_backscatter (namespace backend)'},
 {'id': '[1765269713469, 860555]',
  'time': '2025-12-09T08:41:53.469Z',
  'level': 'info',
  'message': 'Overriding process sar_backscatter (namespace backend)'},
 {'id': '[1765269746664, 188086]',
  'time': '2025-12-09T08:42:26.664Z',
  'level': 'info',
  'message': 'Job spec: {\n "process_graph": {\n  "sarcoherenceparallel1": {\n   "process_id": "sar_coherence_parallel",\n   "arguments": {\n    "InSAR_pairs": [\n     [\n      "2023-07-03",\n      "2023-07-15"\n     ],\n     [\n      "2023-07-15",\n      "2023-07-27"\n     ],\n     [\n      "2023-07-27",\n      "2023-08-08"\n     ],\n     [\n      "2023-08-08",\n      "2023-08-20"\n     ],\n     [\n      "2023-08-20",\n      "2023-09-01"\n     ],\n     [\n      "2023-09-01",\n      "2023-09-13"\n     ],\n     [\n      "2023-09-13",\n      "2023-09-25"\n     ]\n    ],\n    "burst_id": 30343,\n    "coherence_window_az": 5,\n    "coherence_window_rg": 21,\n    "polarization": "vv",\n    "sub_swath": "IW3"\n   }\n  },\n  "saveresult1": {\n   "process_id": "save_result",\n   "arguments": {\n    "data": {\n     "from_node": "sarcoherenceparallel1"\n    },\n    "format": "GTiff",\n    "options": {}\n   },\n   "result": true\n  }\n },\n "job_options": {\n  "image-name": "python311-dev",\n  "log_level": "info"\n }\n}'},
 {'id': '[1765269746709, 571987]',
  'time': '2025-12-09T08:42:26.709Z',
  'level': 'info',
  'message': "ZooKeeperUserDefinedProcessRepository with self._root='/openeo/udps' self._zk_client_reuse=True"},
 {'id': '[1765269748559, 335165]',
  'time': '2025-12-09T08:42:28.559Z',
  'level': 'warning',
  'message': 'Failed to enrich collection metadata of COPERNICUS_VEGETATION_INDICES: 404 Client Error: Not Found for url: https://stac.openeo.vito.be/collections/copernicus_r_utm-wgs84_10_m_hrvpp-vi_p_2017-now_v01'},
 {'id': '[1765269749106, 690959]',
  'time': '2025-12-09T08:42:29.106Z',
  'level': 'warning',
  'message': 'No elastic_job_registry given to GeoPySparkBackendImplementation, creating one'},
 {'id': '[1765269749353, 3351]',
  'time': '2025-12-09T08:42:29.353Z',
  'level': 'info',
  'message': "Loaded config config_id='os_creodias_openeo_k8s_prod' from config_path='/opt/backend_config/backendconfig.py' (reason='lazy_load')"},
 {'id': '[1765269749353, 281667]',
  'time': '2025-12-09T08:42:29.353Z',
  'level': 'warning',
  'message': "No 'openeo_api_version' in `evaluate()` env. Blindly assuming 1.2."},
 {'id': '[1765269749353, 856807]',
  'time': '2025-12-09T08:42:29.353Z',
  'level': 'info',
  'message': 'Correlation id: j-251209082325457ea3e8d47fa023802a'},
 {'id': '[1765269749353, 966262]',
  'time': '2025-12-09T08:42:29.353Z',
  'level': 'info',
  'message': 'Doing dry run'},
 {'id': '[1765269749646, 246430]',
  'time': '2025-12-09T08:42:29.646Z',
  'level': 'warning',
  'message': "[Errno 2] No such file or directory: 'cwltool'"},
 {'id': '[1765269749646, 389643]',
  'time': '2025-12-09T08:42:29.646Z',
  'level': 'warning',
 

In [ ]:
connection.job(job_id).status().get_results().download_files(
    os.path.join(output_folder, connection.job(job_id).describe()['title'][14:])
)